Library Imports


In [1]:
import cv2
import numpy as np
import os
from pathlib import Path
import subprocess

# Path to the folder containing your script
script_dir = os.getcwd() # 

# Build the path relative to that folder - RAW IMAGES FOLDER
address2 = os.path.join(script_dir, "Raw Images", "cam0_capture_20250828_063612.jpg") #cam 0 is the camera behind the Bandpass filter, has less light, needs to be aligned to cam 1
address1 = os.path.join(script_dir, "Raw Images", "cam1_capture_20250828_063612.jpg") #cam 1 is the camera behind the Notch filter, has more light and serves as reference

# # Build the path relative to that folder - NO FILTER data FOLDER
# address2 = os.path.join(script_dir, "NO FILTER data", "cam0_me_lowlight.jpg") #cam 0 is the camera behind the Bandpass filter, has less light, needs to be aligned to cam 1
# address1 = os.path.join(script_dir, "NO FILTER data", "cam1_me_lowlight.jpg") #cam 1 is the camera behind the Notch filter, has more light and serves as reference

# Build the path relative to that folder - RAW VIDEOS FOLDER
vid_address2 = os.path.join(script_dir, "Raw Videos", "cam0_capture_20250828_063612.h264") #cam 0 is the camera behind the Bandpass filter, has less light, needs to be aligned to cam 1
vid_address1 = os.path.join(script_dir, "Raw Videos", "cam1_capture_20250828_063612.h264") #cam 1 is the camera behind the Notch filter, has more light and serves as reference

# # Build the path relative to that folder - RAW VIDEOS FOLDER
# vid_address2 = os.path.join(script_dir, "NO FILTER data", "cam0 record.h264") #cam 0 is the camera behind the Bandpass filter, has less light, needs to be aligned to cam 1
# vid_address1 = os.path.join(script_dir, "NO FILTER data", "cam1 record.h264") #cam 1 is the camera behind the Notch filter, has more light and serves as reference



img1 = cv2.imread(address1)
img2 = cv2.imread(address2)



2 Frames Alignment Function 


In [66]:
# This script aligns two images using feature-based methods.
# It uses ORB feature detection and matching to compute a homography matrix,   
# which is then used to warp one image to align with the other.
# The aligned image is then tested against the original image to ensure alignment.


def align_images_feature_based(img1, img2):
    """
    Align img2 to img1 using ORB feature matching + homography.
    Handles both grayscale and color images.
    """
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(100,100))
    # Ensure grayscale conversion only if needed
    if len(img1.shape) == 3:  # color
        gray1 = clahe.apply(cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY))
    else:  # already grayscale
        gray1 = clahe.apply(img1)

    if len(img2.shape) == 3:  # color
        gray2 = clahe.apply(cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY))
    else:  # already grayscale
        gray2 = clahe.apply(img2)

    # Detect ORB keypoints and descriptors
    orb = cv2.ORB_create(5000)
    kp1, des1 = orb.detectAndCompute(gray1, None)
    kp2, des2 = orb.detectAndCompute(gray2, None)

    # Match features using BFMatcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    if len(matches) < 4:
        raise ValueError("Not enough matches to compute homography.")

    matches = sorted(matches, key=lambda x: x.distance)

    # Extract location of good matches
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Estimate the homography matrix
    M, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

    # Warp img2 to align with img1 (keep same size as img1)
    aligned = cv2.warpPerspective(img2, M, (img1.shape[1], img1.shape[0]))
    
    inliers = mask.sum() if mask is not None else 0

    return aligned, inliers, M


def align_images_with_mirror_check(img1, img2):
    best_aligned = None
    best_inliers = -1

    # Try normal alignment
    try:
        aligned, inliers, M = align_images_feature_based(img1, img2)
        if inliers > best_inliers:
            best_inliers = inliers
            best_aligned = aligned
            best_M = M
    except Exception as e:
        print("Normal alignment failed:", e)

    # Try mirrored alignment
    img2_flipped = cv2.flip(img2, 1)
    try:
        aligned, inliers, M = align_images_feature_based(img1, img2_flipped)
        if inliers > best_inliers:
            best_inliers = inliers
            best_aligned = aligned
            best_M = M
    except Exception as e:
        print("Mirrored alignment failed:", e)

    # if best_aligned is None:
        raise ValueError("Failed to align both normal and mirrored cases.")

    return best_aligned, best_M


def overlay_images_rgb(img1, img2_aligned, alpha=0.5):
    # Resize img2_aligned to match img1 if needed
    if img1.shape != img2_aligned.shape:
        img2_aligned = cv2.resize(img2_aligned, (img1.shape[1], img1.shape[0]))

    # Blend images using alpha
    blended = cv2.addWeighted(img1, alpha, img2_aligned, 1 - alpha, 0)
    return blended



# Align img2 to img1
aligned, M = align_images_with_mirror_check(img1, img2)

# Overlay for visual verification
blended = overlay_images_rgb(img1, aligned)

# Create folder if it doesn't exist
output_folder = "Aligned and Blended images"
os.makedirs(output_folder, exist_ok=True)

# Split into folder, base filename, and extension
base2 = os.path.splitext(os.path.basename(address2))[0]   # cam0_capture_1_20250828_051554
ext2 = os.path.splitext(address2)[1]                      # .jpg
 
# Build new paths
aligned_filename = os.path.join(output_folder, f"{base2}_aligned{ext2}")
blended_filename = os.path.join(output_folder, f"{base2}_blended{ext2}")

cv2.imwrite(aligned_filename, aligned)
cv2.imwrite(blended_filename, blended)

print(f"Saved aligned image to: {aligned_filename}")
print(f"Saved blended image to: {blended_filename}")


Saved aligned image to: Aligned and Blended images\cam0_capture_2_20250828_054449_aligned.jpg
Saved blended image to: Aligned and Blended images\cam0_capture_2_20250828_054449_blended.jpg


2 Frames Focus Assessment Function 

In [ ]:

# Focus loss calculation using variance of Laplacian
def focus_loss(image):
    # If already grayscale, skip conversion
    if len(image.shape) == 2:  # single channel
        gray = image
    else:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    variance = laplacian.var()
    return variance

loss1 = focus_loss(img1)
loss2 = focus_loss(img2)
print(f"Focus loss for img1: {loss1}, img2: {loss2}")
focus_threshold = 0.5  # Example threshold for focus quality
if np.abs(loss1 - loss2) < focus_threshold:
    print("The images are in focus.")
else:
    print("The images are not in focus.")

Focus loss for img1: 5.72860350544188, img2: 12.663590471384262
The images are not in focus.


Video Alignment Function


In [80]:

# ---------------------------
# Helper function to convert & rotate raw H.264
# ---------------------------
def remux_and_rotate_h264(input_file, output_file, fps=25, rotation=None):
    """
    Convert raw H.264 to MP4 with optional rotation (re-encoding required).
    rotation: None, 90, 180, 270 (degrees clockwise)
    """
    if os.path.exists(output_file):
        print(f"MP4 already exists: {output_file}")
        return

    cmd = ["ffmpeg", "-y", "-framerate", str(fps), "-i", input_file]

    # Apply rotation with ffmpeg filters
    if rotation == 90:
        cmd += ["-vf", "transpose=1"]  # 90° clockwise
    elif rotation == 180:
        cmd += ["-vf", "transpose=1,transpose=1"]  # 180° (two 90° clockwise)
    elif rotation == 270:
        cmd += ["-vf", "transpose=2"]  # 90° counter-clockwise (270° clockwise)

    # Re-encode with libx264 to apply rotation
    cmd += ["-c:v", "libx264", "-pix_fmt", "yuv420p", output_file]

    subprocess.run(cmd, check=True)
    print(f"Created MP4: {output_file}")

# ---------------------------
# File paths
# ---------------------------

# Extract base names without .h264 extensions
vid1_base = os.path.splitext(vid_address1)[0]  
vid2_base = os.path.splitext(vid_address2)[0]

# Future mp4 filenames
vid1_mp4 = f"{vid1_base}.mp4"
vid2_mp4 = f"{vid2_base}.mp4"

mp4_base2 = os.path.splitext(os.path.basename(vid2_mp4))[0]
mp4_ext2 = os.path.splitext(vid2_mp4)[1]                      # .mp4

# Create folder if it doesn't exist
output_folder = "Aligned Videos"
os.makedirs(output_folder, exist_ok=True)

output_aligned = os.path.join(output_folder, f"{mp4_base2}_aligned{mp4_ext2}")

# ---------------------------
# Step 1: Convert and rotate raw H.264 to MP4
# ---------------------------
remux_and_rotate_h264(vid_address2, vid2_mp4, fps=25, rotation=180)
remux_and_rotate_h264(vid_address1, vid1_mp4, fps=25, rotation=90)

# ---------------------------
# Step 2: Open MP4s with OpenCV
# ---------------------------
cap0 = cv2.VideoCapture(vid1_mp4)  # reference
cap1 = cv2.VideoCapture(vid2_mp4)  # to align

ret0, frame_ref = cap0.read()
ret1, frame_target = cap1.read()
if not (ret0 and ret1):
    raise ValueError("Failed to read the first frames from the videos.")

h, w = frame_ref.shape[:2]
fps = cap0.get(cv2.CAP_PROP_FPS) or 25
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_aligned, fourcc, fps, (w, h))

# Reset to first frame
cap0.set(cv2.CAP_PROP_POS_FRAMES, 0)
cap1.set(cv2.CAP_PROP_POS_FRAMES, 0)

# ---------------------------
# Step 3: Frame-by-frame alignment
# ---------------------------

ret0, frame_ref = cap0.read()
ret1, frame_target = cap1.read()
if not (ret0 and ret1):
     raise ValueError("Failed to read frames for homography estimation.")

# Align cam1 to cam0 using first frame to get homography
aligned_frame, M = align_images_with_mirror_check(frame_ref, frame_target)

# Reset both videos to frame 0 again
cap0.set(cv2.CAP_PROP_POS_FRAMES, 0)
cap1.set(cv2.CAP_PROP_POS_FRAMES, 0)

# ---------------------------
# Step 4: Apply homography to all frames
# ---------------------------
while True:
    ret0, frame_ref = cap0.read()
    ret1, frame_target = cap1.read()
    frame_target = cv2.flip(frame_target, 1)  # Flip frame_target horizontally
    if not (ret0 and ret1):
        break

    # Apply the SAME homography to every frame of cam1
    aligned_frame = cv2.warpPerspective(frame_target, M, (w, h))

    out.write(aligned_frame)
# ---------------------------
# Step 5: Cleanup
# ---------------------------
cap0.release()
cap1.release()
out.release()

print("Alignment complete. Output saved to:", output_aligned)


MP4 already exists: c:\Users\liats\sodium-vapor-camera-pi-5-code\Raw Videos\cam0_capture_20250828_063612.mp4
MP4 already exists: c:\Users\liats\sodium-vapor-camera-pi-5-code\Raw Videos\cam1_capture_20250828_063612.mp4
Alignment complete. Output saved to: Aligned Videos\cam0_capture_20250828_063612_aligned.mp4


Image Compositing

In [5]:
def sodium_vapor_composite_from_paths(fg, bg, matte, save,
                                      output_folder="Composited Results"):
    """
    Sodium vapor compositing: combines foreground, background, and matte images.
    Accepts either:
        - File paths (strings)
        - Already-loaded frames (NumPy arrays)
    save: whether to save the output (used for still images; ignored for video frames)
    Returns composited, fg_only, bg_only, matte, paths   """
    os.makedirs(output_folder, exist_ok=True)

    # If input is a path, load image
    if isinstance(fg, str):
        foreground_color = cv2.imread(fg)
        background_image = cv2.imread(bg)
        sodium_matte = cv2.imread(matte, cv2.IMREAD_GRAYSCALE)
        save_flag = save
    else:
        foreground_color, background_image, sodium_matte = fg, bg, matte
        save_flag = False  # never save when passing frames

    
    # Sanity check
    if foreground_color is None or background_image is None or sodium_matte is None:
        raise ValueError("One or more image paths are invalid or files cannot be read.")

    # --- Prepare sizes ---
    h, w = foreground_color.shape[:2]
    if background_image.shape[:2] != (h, w):
        background_image = cv2.resize(background_image, (w, h))
    if sodium_matte.shape[:2] != (h, w):
        sodium_matte = cv2.resize(sodium_matte, (w, h))

    
   # Resize bg and matte to match fg
    background_image = cv2.resize(background_image, (foreground_color.shape[1], foreground_color.shape[0]))
    if len(sodium_matte.shape) == 3:  # If matte is color, convert to grayscale
        sodium_matte = cv2.cvtColor(sodium_matte, cv2.COLOR_BGR2GRAY)
    sodium_matte = cv2.resize(sodium_matte, (foreground_color.shape[1], foreground_color.shape[0]))

    # Normalize matte to 0..1 and make 3-channel
    # matte_norm = sodium_matte.astype("float32") / 255.0
    matte_norm = cv2.normalize(sodium_matte.astype("float32"), None, 0.0, 1.0, cv2.NORM_MINMAX)
    matte_3ch = cv2.merge([matte_norm]*3)
    inv_matte_3ch = 1.0 - matte_3ch

    # Extract parts
    fg_part = (foreground_color.astype(np.float32) * matte_3ch).astype(np.uint8)
    fg_part = cv2.convertScaleAbs(fg_part, alpha=5, beta=10)  # slightly darker foreground for better blending


    bg_part = (background_image.astype(np.float32) * inv_matte_3ch).astype(np.uint8)
    # bg_part = cv2.convertScaleAbs(bg_part, alpha=5.5, beta=80)  # Brighten background slightly

    composited = cv2.add(fg_part, bg_part)

    if save: # Save results only if save is True (when compositing pics, not videos)
        output_folder = "Composited Results"
        os.makedirs(output_folder, exist_ok=True)

        # Determine base names for saving only if input is a path
        if isinstance(fg, str):
            fg_base = os.path.splitext(os.path.basename(fg))[0]
        else:
            fg_base = "fg_frame"

        if isinstance(bg, str):
            bg_base = os.path.splitext(os.path.basename(bg))[0]
        else:
            bg_base = "bg_frame"
        if isinstance(matte, str):
            matte_base = os.path.splitext(os.path.basename(matte))[0]
        else:
            matte_base = "matte_frame"

        fg_only_path = os.path.join(output_folder, f"{fg_base}_fg_only.jpg")
        bg_only_path = os.path.join(output_folder, f"{bg_base}_bg_only_{matte_base}.jpg")
        matte_only_path = os.path.join(output_folder, f"{matte_base}_matte_only.jpg")

        composited_path = os.path.join(output_folder, f"{fg_base}_{bg_base}_composited.jpg")

        cv2.imwrite(fg_only_path, fg_part)
        cv2.imwrite(bg_only_path, bg_part)
        cv2.imwrite(matte_only_path, matte_3ch*255)
        cv2.imwrite(composited_path, composited)

        print(f"✅ Saved: {fg_only_path}, {bg_only_path}, {matte_only_path}, {composited_path}")

        return composited, fg_part, bg_part, matte_3ch, (fg_only_path, bg_only_path, matte_only_path, composited_path)
    else:
        return composited, fg_part, bg_part, matte_3ch, None

# Load images
fg_address = "Raw Images/cam1_capture_1_20250828_051554.jpg"   # actors - the "normal" camera image with the actors (what you want to keep).
bg_address = "Backgrounds/beach.jpg"  # scenery - the replacement scenery (the plate you want to paste the actors onto).

matte_address = "Aligned and Blended images/cam0_capture_1_20250828_051554_aligned.jpg"  # sodium channel matte - the sodium-sensitive camera image 
# (basically a mask of just the background — it looks like a silhouette of your actors in black with the sodium-lit screen in white).
# must match the picture of the foreground_color image (but with a different camera). 

composited, fg_only, bg_only, matte, paths = sodium_vapor_composite_from_paths(fg_address, bg_address, matte_address, save=True)

print("✅ Composite saved in 'Composited Results' folder")


✅ Saved: Composited Results\cam1_capture_1_20250828_051554_fg_only.jpg, Composited Results\beach_bg_only_cam0_capture_1_20250828_051554_aligned.jpg, Composited Results\cam0_capture_1_20250828_051554_aligned_matte_only.jpg, Composited Results\cam1_capture_1_20250828_051554_beach_composited.jpg
✅ Composite saved in 'Composited Results' folder


Video Compositing


In [ ]:

os.makedirs("Composited Results", exist_ok=True)

# Inputs
fg_cap = cv2.VideoCapture("Raw Videos/cam1_capture_20250828_063612.mp4")
bg_path = "Backgrounds/Zisapel-Building.jpg"   # <- can be video OR image
bg_base = os.path.splitext(os.path.basename(bg_path))[0]

matte_cap = cv2.VideoCapture("Aligned Videos/cam0_capture_20250828_063612_aligned.mp4")

bg_cap = cv2.VideoCapture(bg_path)

# How many frames does it report?
frame_count = int(bg_cap.get(cv2.CAP_PROP_FRAME_COUNT))

if frame_count <= 1:
    # Treat as still image
    ret, frame = bg_cap.read()
    if not ret:
        raise ValueError(f"Could not read background: {bg_path}")
    bg_image = frame
    bg_cap.release()
    bg_cap = None
    bg_is_video = False
else:
    bg_image = None
    bg_is_video = True


# Setup writer
fps = fg_cap.get(cv2.CAP_PROP_FPS)
w = int(fg_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(fg_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

composited_path = os.path.join("Composited Results", f"final_composite_{bg_base}.mp4")
foreground_path = os.path.join("Composited Results", f"final_foreground_{bg_base}.mp4")
background_path = os.path.join("Composited Results", f"final_background_{bg_base}.mp4")
matte_path = os.path.join("Composited Results", f"final_matte_{bg_base}.mp4")

out = cv2.VideoWriter(composited_path, fourcc, fps, (w, h))
out_fg = cv2.VideoWriter(foreground_path, fourcc, fps, (w, h))
out_bg = cv2.VideoWriter(background_path, fourcc, fps, (w, h))
out_matte = cv2.VideoWriter(matte_path, fourcc, fps, (w, h))

frame_idx = 0
while True:
    ret_fg, fg_frame = fg_cap.read()
    ret_matte, matte_frame = matte_cap.read()

    if not (ret_fg and ret_matte):
        break

    # Handle background
    if bg_is_video:
        ret_bg, bg_frame = bg_cap.read()
        if not ret_bg:
            break
    else:
        bg_frame = cv2.resize(bg_image, (w, h))  # reuse same still image every frame

    # Convert matte to grayscale
    matte_gray = cv2.cvtColor(matte_frame, cv2.COLOR_BGR2GRAY)

    # Reuse compositing function
    composited, fg_only, bg_only, matte, paths = sodium_vapor_composite_from_paths(
        fg_frame, bg_frame, matte_gray, save=False
    )

    out.write(composited)
    out_fg.write(fg_only)
    out_bg.write(bg_only)
    out_matte.write((matte*255).astype(np.uint8))
    frame_idx += 1
    if frame_idx % 50 == 0:
        print(f"Processed {frame_idx} frames...")

fg_cap.release()
if bg_is_video:
    bg_cap.release()
matte_cap.release()
out.release()
out_fg.release()
out_bg.release()
out_matte.release()

print("✅ Video compositing complete.")


ValueError: too many values to unpack (expected 4)